Universidade Federal de Alagoas

Instituto de Computação


Disciplina: Sistemas Distribuídos

Semestre letivo: 2018.1

Professor: André Lage Freitas


# Trabalho Prático AB2

**Importante**. Leia atentamente as especificações pois só serão aceitos trabalhos que as obedecerem.


## Parte 1: Implementação de um monitor de caraga da trabalho descentralizado

**Implementar uma versão modificada do detector de falha não confiável** (_unreliable failure detector_) descrito na Seção _15.1.1 Failure assumptions and failure detectors_ do livro a seguir:

Coulouris, Dollimore, Kindberg and Blair, _Distributed Systems: Concepts and Design_. Addison-Wesley, 2012.

A **modificação** consiste em enviar informações sobre a carga do sistema (_system workload_) em vez de informações sobre falhas que indicariam os processos suspeitos e não-suspeitos. Para isso, em vez de cada processo enviar mensagem do tipo `I'm p and I'm alive`, os processos enviarão a medição interna da sua carga de trabalho. 

Para simular a medição da carga de trabalho dos processos, utilize números aleatórios entre `0` e `1` com precisão de uma casa decimal apenas, por exemplo, `0.4`.

Utilize como período de envio das mensagens `2` segundos.

### Detalhes de implementação

Cada processo do sistema deverá ser implementado como um Julia Worker. O Master será apenas o gestor do experimento, ou seja, será responsável para criar os processos (Julia Workers) e iniciar o sistema. Uma vez iniciado, o sistema deve funcionar por si sem depender do Master; uma vez que o algoritmo é descentralizado.

Para fins de visualização do monitoramento da carga de trabalho, os **Workers devem enviar ao Master** periodicamente (a cada `2` segundos) uma mensagem com o estado dos workers que eles monitoram (incluindo ele mesmo). Essa mensagem deve ser impresso na tela do master da seguinte forma.

```julia
[...]
Workload from Worker 4: [0.4,0.9,0.7]
Workload from Worker 2: [0.4,0.9,0.7]
Workload from Worker 3: [0.4,0.9,0.7]
[...]
```

A última linha acima significa que as cargas de trabalho dos Workers monitorados pelo Worker 3 são `0.4,0.9,0.7`, ou seja, os Workers têm as seguintes carga de trabalho:

* Worker 2 = 0.4
* Worker 3 = 0.9
* Worker 4 = 0.7

Lembre-se de que, em Julia, o ID do Master é 1 e os Workers têm ID maiores que 1.

## Parte 2: Implementação de um algoritmo de Eleição

**Implementar o algoritmo de Eleição** baseado em anel (_ring-based election algorithm_) descrito na Seção _15.3 Elections_ do livro a seguir:

Coulouris, Dollimore, Kindberg and Blair, _Distributed Systems: Concepts and Design_. Addison-Wesley, 2012.

Considere as seguintes condições.

1. Uma eleição nova é disparada **quando a carga de trabalho do líder for maior ou igual a `0.8`** e **o Worker que identificou essa alta carga de trabalho do líder tenha carga de trabalho menor que `0.8`**.
1. Em caso de empate, o Worker com ID mais alto ganha a eleição.
1. Caso não haja Worker com carga de trabalho menor que `0.8` para liderar, escolhe-se o Worker com menor carga de trabalho.

Quando um líder for eleito, ele deve enviar a a seguinte mensagem ao Master:

```julia
[...]
New leader elected! Worker 2
[...]
```


### Suporte de programação distribuída em Julia

Para utilizar o suporte de programação distribuída em Julia, primeiro devemos carergar a biblioteca `Distributed`:

```julia
using Pkg
Pkg.add("Distributed")
using Distributed
```

A seguir, segue uma lista (não extensiva) de funções que poderão ajudar na programação distribuída para a implementação do trabalho:

```julia
addprocs
remotecall
@fetchfrom
@everywhere
sleep
@spawn
```

Para buscar uma rápida ajuda sobre as funções, utilize `?` na frente da função (ver exemplo abaixo). 

```julia
?addprocs
```

Utilize o [material do mini-curso do Professor](https://github.com/proflage/2018-julia-hands-on) para estudo e a [documentação oficial da linguagem Julia (1.0)](https://docs.julialang.org/en/stable/) para demais dúvidas.

## Código-fonte

Documente seu código em comentários no próprio código-fonte.

**Todo o seu código tem que estar colado na próxima célula**, mesmo se ele tenha sido feito em IDEs, células separadas, etc. O Professor testará a apenas o que estiver colado na próxima célula.

In [ ]:
#=
    The following code was written by Júlio "Jooj" de Holanda, for the
    Distributed Systems course of the Computer Science course of the
    Universidade Federal de Alagoas (UFAL), taught by Professor André Lage.
=#

using Pkg
#Pkg.add("Distributed")
using Distributed

qntWorkers = 4  #Defining how many workers the system will have
Distributed.addprocs(qntWorkers)    #Creating the workers

#=
    These are the distributed variables that system workers can access.
    Each variable is an array and each position of each array can only be accessed
    by its respective worker.

    For example, positions 2 of the arrays can only be accessed by worker 2.
=#

@everywhere workloads = fill(0.0, length(workers()))    #The workloads of each worker
@everywhere leaderId = fill(2, length(workers())) #The id of the current leader
@everywhere candidateId = fill(-1, length(workers()))   #The id of the leader candidate chosen by each worker
@everywhere candidateWorkload = fill(1.0, length(workers()))    #The workload of each worker as a vote for their candidate to leader

### Master-side functions ###

# Receives the workloads of a worker and the monitored workers and displays them on the screen
function receiveDataFromWorker(message)
    println(message)
end

### Worker-side functions ###

#=
    It is looped by a worker.
    Updates the current workload of the worker, calls the function responsible
    for updating the workloads of the other workers and sends the updated loads
    to the master.
=#
@everywhere function failureDetection()
    myid = Distributed.myid()

    while true
        workloads[myid-1] = round(rand(1)[1], digits = 1)
        for pid in workers()
            if pid != myid
                @spawnat pid updateWorkloads(myid, workloads[myid-1])
            end
        end
        sleep(2)

        message = "Workload from Worker $(myid): $(workloads)"
        @spawnat 1 receiveDataFromWorker(message)
    end
end

#=
    Receives the workloads of another worker to update the load list.
    If the worker you sent is the leader and the received load was greater than
    or equal to 0.8, an election may be initiated.
=#
@everywhere function updateWorkloads(workerId, workerWorkload)
    myid = Distributed.myid()

    if candidateId[myid-1] == -1    #If an election was not started by this worker, then run the code below
    workloads[workerId-1] = workerWorkload

        #=
            If the received load came from the leader and is greater than or
            equal to 0.8 and the current worker's workload is less than 0.8
            then a new election is initiated
        =#
        if workerId == leaderId[myid-1] && workerWorkload >= 0.8 && workloads[myid-1] < 0.8
            startElection()
        end
    end
end

#=
    Updates the current leader with the id of a new leader. Also restores the
    original values of the candidateId and candidateWorkloads arrays.
=#
@everywhere function updateLeader(newLeaderId)
    for pid in workers()
        leaderId[pid-1] = newLeaderId
        candidateId[pid-1] = -1
        candidateWorkload[pid-1] = 1.0
    end
end

# Defines the vote of the worker and his candidate, sending to the next worker
@everywhere function startElection()
    myid = Distributed.myid()

    candidateId[myid-1] = myid
    candidateWorkload[myid-1] = workloads[myid-1]
    nextId = myid+1

    #=
        If the worker is the last in the list of ids, then the list goes back
        to the beginning, in this case worker 2.
    =#
    if nextId > length(workers())+1
        nextId = 2
    end
    @spawnat nextId election(candidateId[myid-1], candidateWorkload[myid-1])    #Send the vote and the candidate to the next worker
end

#=
    Receive the candidate and the vote of the previous worker. If the votes are
    equal, the candidate is the worker with the most id. If not, the candidate
    will be the one with the lowest workload value (vote).
=#
@everywhere function election(workerId, workerWorkload)
    myid = Distributed.myid()
    myWorkload = workloads[myid-1]
    nextId = myid+1

    if myid == workerId # If the id is the same, then the election is over
        for pid in workers()    # Update the leader in all workers
            @spawnat pid updateLeader(myid)
        end
        println("New leader elected! Worker $(leaderId[myid-1])")
    else
        if myid != leaderId[myid-1] # Id the worker is the current leader, then skip to the next worker
            if myWorkload < workerWorkload
                candidateId[myid-1] = myid
                candidateWorkload[myid-1] = myWorkload
            elseif myWorkload > workerWorkload
                candidateId[myid-1] = workerId
                candidateWorkload[myid-1] = workerWorkload
            else
                if myid > workerId
                    candidateId[myid-1] = myid
                    candidateWorkload[myid-1] = myWorkload
                else
                    candidateId[myid-1] = workerId
                    candidateWorkload[myid-1] = workerWorkload
                end
            end
        else
            if myWorkload >= 0.8
                candidateId[myid-1] = workerId
                candidateWorkload[myid-1] = workerWorkload
            else
                candidateId[myid-1] = myid
                candidateWorkload[myid-1] = myWorkload
            end
        end

        #=
            If the worker is the last in the list of ids, then the list goes back
            to the beginning, in this case worker 2.
        =#
        if nextId > length(workers())+1
            nextId = 2
        end
        @spawnat nextId election(candidateId[myid-1], candidateWorkload[myid-1])    #Send the vote and the candidate to the next worker
    end
end

### Init system ###

for pid in workers()    # Call failureDetection in all workers
    @spawn failureDetection()
end



## Entrega

As respostas deverão ser entregues na parte indicada desse arquivo reservada ao código fonte, no formato [IJulia Notebook](https://github.com/JuliaLang/IJulia.jl), que utiliza tecnologia [Jupyter](https://www.jupyter.org).

O programa deve ser implementado na linguagem de programação [**Julia** versão 1.0](https://docs.julialang.org/en/stable/). Seu trabalho será testado na [JuliaBox](https://juliabox.com). 

Baixe seu arquivo `.ipynb` e anexe-o ao Google Classroom. Não se esqueça de **testar seu arquivo `.ipynb` IJulia na JuliaBox** antes de enviá-lo.


### Forma 

O arquivo IJulia Notebook deverá ser entregue ao Professor **exclusivamente através do Google Classroom**.



A responsabilidade sobre a integridade do arquivo contendo trabalho é exclusivamente dos discentes. Serão ignorados os trabalhos cujos arquivos não conseguirem ser abertos pelo Professor.

### Prazos

O prazo de entrega está descrito no **Google Classroom**.


## Pontuação extra

O(a) discente que realizar mais tarefas, além do que foi especificado neste trabalho, o professor atribuirá de 0,5 a 1,0 ponto extra a depender da relevância da contribuição no programa. O critério será decidido pelo professor. 

O(a) discente deve indicar qual é a tarefa executada. Por exemplo, adição de funcionalidades, armazenamento de dados em arquivo, documentação de código, comentários em inglês sem erros ortográficos, etc.


**Plágio** A nota zero será atribuída caso haja qualquer tipo de cópia parcial ou integral assim como as devidas medidas legais. Leia a [cartilha sobre plágio](http://www.noticias.uff.br/arquivos/cartilha-sobre-plagio-academico.pdf).

## Disclaimer

Esse material foi elaborado pelo [Prof. André Lage Freitas](https://sites.google.com/a/ic.ufal.br/andrelage/) e está licenciado sob a licença _GNU General Public License v3.0_.